<a href="https://colab.research.google.com/github/GuanRuLai/Python-project-Youtube-Video-Comment-Chatbot/blob/main/Youtube_video_comment_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries

In [1]:
!pip install langchain
!pip install langchain_openai rich
!pip install youtube_search
!pip install yt-dlp
!pip install faiss-gpu
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


# Import libraries

In [2]:
import os
from google.colab import userdata
from langchain_community.tools import YouTubeSearchTool
from yt_dlp import YoutubeDL
from openai import OpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import MessagesPlaceholder
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Set api key

In [3]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# define a function for getting the url of the video

In [4]:
def search_url(query):
  tool = YouTubeSearchTool() # Build Youtube tool object
  result = tool.invoke(query + ",1")
  urls = eval(result) # Parse the string into a Python expression
  urls = [url.split("&")[0] for url in urls]

  return urls

# Define a function for downloading the the audio of the video

In [5]:
def url_download(url):
    try:
        # Set output file name
        drive_path = "/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI"
        output_path = os.path.join(drive_path, "audio")

        # yt-dlp options for downloading audio
        ydl_opts = {
            'format': 'bestaudio/best', # Download the best quality audio
            'outtmpl': output_path,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio', # Extract audio
                'preferredcodec': 'mp3', # Convert to mp3
                'preferredquality': '192', # Audio quality
            }],
        }

        # Download the audio
        with YoutubeDL(ydl_opts) as ydl:
            print(f"開始下載: {url}")
            ydl.download([url])
            print("音訊下載成功")
    except Exception as e:
        print(f"下載過程中出現錯誤: {e}")

# Define a function for ASR using OpenAI Whisper API

In [6]:
def audio_to_text(file_path):
  client = OpenAI()
  audio_file = open(file_path, "rb")
  transcript = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file
    )

  return transcript.text

# Define a function for data processing & RAG chain building

In [8]:
def rag(text):
  # split the data
  text_splitter = RecursiveCharacterTextSplitter(separators=[" "], # Slice on a word basis
                                               chunk_size=300, # Maximum size of each cut text block
                                               chunk_overlap=20) # Overlap between adjacent text blocks(make sure there is some context)
  splits = text_splitter.split_text(text)

  # convert text to vector & store into vector database
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
  db = FAISS.from_texts(splits, embeddings)
  db.save_local("/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/youtube_db")

  new_db = FAISS.load_local(
      folder_path="/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/youtube_db",
      embeddings=embeddings,
      allow_dangerous_deserialization=True)

  chat_model = ChatOpenAI(model="gpt-4o-mini")

  str_parser = StrOutputParser()

  # Creates a retriever tool for fetching relevant video content based on queries
  retriever = new_db.as_retriever()

  tool = create_retriever_tool(
      retriever=retriever,
      name="retriever_tool",
      description="檢索並返回文字相關資訊"
  )

  tools = [tool]

  # Create agent
  agent_prompt = ChatPromptTemplate.from_messages([
      ("system", "你是一位善用工具的好助理, 請仔細判斷上下文來回答問題, 盡量以文字內容 RAG 功能優先"),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
      MessagesPlaceholder(variable_name="agent_scratchpad")
  ])

  agent = create_openai_tools_agent(llm=chat_model,
                                    tools=tools,
                                    prompt=agent_prompt)

  agent_executor = AgentExecutor(agent=agent,
                                 tools=tools,
                                 verbose=False) # The system won't show whether it use the tool or not

  # Store chat messages history
  memory = SQLChatMessageHistory(
      session_id="test_id",
      connection_string="sqlite:////content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/retriever.db"
  )

  # Limit the chat memory to the latest six messages for context clarity
  def window_messages(chain_input):
      if len(memory.messages) > 6:
          cur_messages = memory.messages
          memory.clear()
          for message in cur_messages[-6:]:
              memory.add_message(message)
      return None

  # Integrate chat history into the agent's execution for consistent interaction
  def add_history(agent_executor):
      agent_with_chat_history = RunnableWithMessageHistory(
          agent_executor,
          lambda session_id: memory,
          input_messages_key="input",
          history_messages_key="chat_history",
      )
      memory_chain = (
          RunnablePassthrough.assign(messages=window_messages)
          | agent_with_chat_history
      )
      return memory_chain

  return add_history(agent_executor)

# Main program

In [11]:
while True:
    msg = input("請問您要查詢的影片關鍵字是？：")
    if not msg.strip():
        break

    urls = search_url(msg)
    print(urls)
    url_download(urls[0])
    print("處理中，請稍後...")
    text = audio_to_text("/content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/audio.mp3")
    print("處理完畢，可以開始進行問答！")

    memory_chain = rag(text)
    if memory_chain is None:
      print("無法建立記憶鏈，請檢查輸入資料。")
      continue

    while True:
        msg = input("我的問題：")
        if not msg.strip():
            break
        for chunk in memory_chain.stream(
            {"input": msg},
            config={"configurable": {"session_id": "test_id"}}):
            if "output" in chunk:
                print(f"AI 回覆：{chunk['output']}", end="")

請問您要查詢的影片關鍵字是？：邱軍
['https://www.youtube.com/watch?v=DIKkgjxnzRk']
開始下載: https://www.youtube.com/watch?v=DIKkgjxnzRk
[youtube] Extracting URL: https://www.youtube.com/watch?v=DIKkgjxnzRk
[youtube] DIKkgjxnzRk: Downloading webpage
[youtube] DIKkgjxnzRk: Downloading ios player API JSON
[youtube] DIKkgjxnzRk: Downloading mweb player API JSON
[youtube] DIKkgjxnzRk: Downloading m3u8 information
[info] DIKkgjxnzRk: Downloading 1 format(s): 251
[download] Destination: /content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/audio
[download] 100% of    1.56MiB in 00:00:00 at 15.41MiB/s  
[ExtractAudio] Destination: /content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/audio.mp3
Deleting original file /content/drive/MyDrive/Colab Notebooks/人工智慧-深度學習/深度學習模型實作/生成式 AI/audio (pass -k to keep)
音訊下載成功
處理中，請稍後...
處理完畢，可以開始進行問答！
我的問題：請問剛剛下載的內容是關於什麼？
AI 回覆：剛剛下載的內容是關於一起交通事故的報導，涉及歌手邱君。事件發生在去年12月27日，邱君駕駛的車輛在基隆市撞上了一名計程車司機，該司機在搶救一週後宣告死亡。報導中提到事故後邱君未停車並拒絕接聽電話，最終兩天後才現身，並表示因忙碌無法及時到案。警方已展開調查，並將邱